<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-393 Máquinas de Aprendizaje II-2018 </h1>

<H3 align='center'> Tarea 0 - Introducción a Máquinas de Aprendizaje </H3>
<hr style="height:2px;border:none"/>

Integrantes:
- Diego Jara - 
- Miguel Ibáñez - 2990010-8

In [2]:
import re, time
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import mean_absolute_error #measure MAE

from nltk.corpus import stopwords
from nltk import WordNetLemmatizer, word_tokenize
from nltk.stem.porter import PorterStemmer

## 2. Job Salary Prediction

** a) Lectura de data y sets **

In [5]:
df = pd.read_csv("Train_rev1.csv")

df_train = df.iloc[:-20000] # Train set
df_val = df.iloc[-10000:] # Validation set
df_test = df.iloc[-20000:-10000] # Test set

msk = np.random.rand(len(df_train)) < 0.1 # Sample del train dataset
df_train_subset = df_train[msk]

In [6]:
print("Train dataset shape:{}".format(df_train_subset.shape))
print("Test dataset shape:{}".format(df_test.shape))
print(df_train_subset.info())

Train dataset shape:(22692, 12)
Test dataset shape:(10000, 12)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 22692 entries, 0 to 224764
Data columns (total 12 columns):
Id                    22692 non-null int64
Title                 22692 non-null object
FullDescription       22692 non-null object
LocationRaw           22692 non-null object
LocationNormalized    22692 non-null object
ContractType          6283 non-null object
ContractTime          16573 non-null object
Company               19445 non-null object
Category              22692 non-null object
SalaryRaw             22692 non-null object
SalaryNormalized      22692 non-null int64
SourceName            22692 non-null object
dtypes: int64(2), object(10)
memory usage: 2.3+ MB
None


In [7]:
df_train_subset.head()

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 40000/annum 20-40K,30000,cv-library.co.uk
8,14663197,RECRUITMENT CONSULTANT CONSTRUCTION / TECHNIC...,This is an exceptional opportunity to join a c...,"LEEDS, West Yorkshire",Leeds,NaN,permanent,Code Blue Recruitment,HR & Recruitment Jobs,18000 - 28000/annum 18 - 28K BASIC + COMM + BENS,23000,cv-library.co.uk
14,20797143,Chef de Partie Award Winning Restaurant Exce...,A well respected restaurant located in the Der...,Derby Derbyshire East Midlands,Derby,NaN,NaN,Chef Results,Hospitality & Catering Jobs,"Upto 16,000 per annum + Tips (IRO 3K Per Annum)",16000,caterer.com


**b) Extracción de datos**

In [8]:
txt_raw_train = df_train_subset.FullDescription
txt_raw_val = df_val.FullDescription
txt_raw_test = df_test.FullDescription

salary_train = np.asarray(df_train_subset["SalaryNormalized"])
salary_val = np.asarray(df_val["SalaryNormalized"])
salary_test = np.asarray(df_test["SalaryNormalized"])

**c) Pre-procesamiento**

In [9]:
def word_extractor(text):
    wordstemmer = PorterStemmer()
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1',text)#substitute multiple letter by two
    words = ""
    wordtokens = [ wordstemmer.stem(word.lower())for word in word_tokenize(text) ]
    for word in wordtokens:
        if word not in commonwords:
            words+=" "+word
    return words

In [11]:
texts_train = [word_extractor(text) for text in txt_raw_train]
texts_val = [word_extractor(text) for text in txt_raw_val]

vectorizer1 = CountVectorizer()
vectorizer1.fit(np.asarray(texts_train))

features_train = vectorizer1.transform(texts_train)
features_val = vectorizer1.transform(texts_val)

**d) Regresión**

In [12]:
model = LR(fit_intercept=True, normalize=False)
model.fit(features_train,salary_train)

LR_fo = lambda x,y: sum((x-y)**2)
print("F.O. on train: ",LR_fo(salary_train, model.predict(features_train)))
print("F.O. on validation: ",LR_fo(salary_val, model.predict(features_val)))
print()

print("MAE on train: ",mean_absolute_error(salary_train, model.predict(features_train)))
print("MAE on validation: ",mean_absolute_error(salary_val, model.predict(features_val)))

F.O. on train:  3158684806.9
F.O. on validation:  8.75063838217e+13

MAE on train:  21.0737930613
MAE on validation:  63752.3262719
